In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dd-dream-team/dd_score_analysis/598067.yaml
/kaggle/input/dd-dream-team/dd_score_analysis/419155.yaml
/kaggle/input/dd-dream-team/dd_score_analysis/392183.yaml
/kaggle/input/dd-dream-team/dd_score_analysis/548310.yaml
/kaggle/input/dd-dream-team/dd_score_analysis/336022.yaml
/kaggle/input/dd-dream-team/dd_score_analysis/501265.yaml
/kaggle/input/dd-dream-team/dd_score_analysis/548357.yaml
/kaggle/input/dd-dream-team/dd_score_analysis/598062.yaml
/kaggle/input/dd-dream-team/dd_score_analysis/392237.yaml
/kaggle/input/dd-dream-team/dd_score_analysis/392206.yaml
/kaggle/input/dd-dream-team/dd_score_analysis/548329.yaml
/kaggle/input/dd-dream-team/dd_score_analysis/419111.yaml
/kaggle/input/dd-dream-team/dd_score_analysis/598044.yaml
/kaggle/input/dd-dream-team/dd_score_analysis/392230.yaml
/kaggle/input/dd-dream-team/dd_score_analysis/548364.yaml
/kaggle/input/dd-dream-team/dd_score_analysis/501253.yaml
/kaggle/input/dd-dream-team/dd_score_analysis/501216.yaml
/kaggle/input/

In [2]:
# importing libraries
import math
import yaml


In [3]:
# path to uplaoded zip file

data_path = "/kaggle/input/dd-dream-team/dd_score_analysis"

# List YAML files
yaml_files = [os.path.join(data_path, f) for f in os.listdir(data_path) if f.endswith(".yaml")]
print(f"Total YAML files: {len(yaml_files)}")

Total YAML files: 92


In [4]:
# load first file to inspect the structure
with open(yaml_files[0])as file:
    match = yaml.safe_load(file)
print(match.keys())

dict_keys(['meta', 'info', 'innings'])


Extracting the data from the new yaml file


In [181]:
all_data = []
for file in yaml_files:
    with open(file, 'r') as f:
        match = yaml.safe_load(f)

    match_info = match['info']
    match_id = f"{match_info['dates'][0]}, {match_info['venue']}"
    teams = match['info']['teams']

    for inning in match['innings']:
        inning_name = list(inning.keys())[0]
        inning_data = inning[inning_name]
        deliveries = inning_data['deliveries']
        inning_number = 1 if inning_name == '1st innings' else 2
        batting_team = inning_data['team']
        bowling_team = teams[1] if teams[0] == batting_team else teams[0]
    
        for delivery in deliveries:
            wicket_info = delivery.get('wicket', {})
            dismissal_kind = wicket_info.get('kind', None)
            ball_no = list(delivery.keys())[0]
            delivery_data = delivery[ball_no]
            wicket_info = delivery_data.get('wicket', {})
            
            row = {
                "match_id": match_id,
                "inning": inning_number,
                "over": int(str(ball_no).split('.')[0]),
                "ball": float(ball_no),
                "batting_team": batting_team,
                "bowling_team": bowling_team,
                "batsman": delivery_data.get("batsman"),
                "non_striker": delivery_data.get("non_striker"),
                "bowler": delivery_data.get("bowler"),
                "runs_batsman": delivery_data["runs"]["batsman"],
                "runs_extras": delivery_data["runs"]["extras"],
                "runs_total": delivery_data["runs"]["total"],
                "dismissal_kind": dismissal_kind,
                "player_out": wicket_info.get("player_out", None)
            }
    
            all_data.append(row)

# Convert to DataFrame
initial_df = pd.DataFrame(all_data)


In [183]:
initial_df.tail()

,match_id,inning,over,ball,batting_team,bowling_team,batsman,non_striker,bowler,runs_batsman,runs_extras,runs_total,dismissal_kind,player_out
21517,"2008-05-11, Sawai Mansingh Stadium",2,18,18.3,Rajasthan Royals,Delhi Daredevils,M Rawat,Niraj Patel,MF Maharoof,0,0,0,None,None
21518,"2008-05-11, Sawai Mansingh Stadium",2,18,18.4,Rajasthan Royals,Delhi Daredevils,M Rawat,Niraj Patel,MF Maharoof,0,0,0,None,None
21519,"2008-05-11, Sawai Mansingh Stadium",2,18,18.5,Rajasthan Royals,Delhi Daredevils,M Rawat,Niraj Patel,MF Maharoof,1,0,1,None,None
21520,"2008-05-11, Sawai Mansingh Stadium",2,18,18.6,Rajasthan Royals,Delhi Daredevils,Niraj Patel,M Rawat,MF Maharoof,4,0,4,None,None
21521,"2008-05-11, Sawai Mansingh Stadium",2,19,19.1,Rajasthan Royals,Delhi Daredevils,M Rawat,Niraj Patel,VY Mahesh,4,0,4,None,None


In [169]:
initial_df[['batting_team', 'bowling_team']].drop_duplicates().head()

,batting_team,bowling_team
0,Pune Warriors,Delhi Daredevils
123,Delhi Daredevils,Pune Warriors
250,Chennai Super Kings,Delhi Daredevils
376,Delhi Daredevils,Chennai Super Kings
492,Kings XI Punjab,Delhi Daredevils


Now, that we have flat data we will do some EDA and inspect the data further.

In [170]:
initial_df.loc[10]

match_id          2013-05-19, Subrata Roy Sahara Stadium
inning                                                 1
over                                                   1
ball                                                 1.5
batting_team                               Pune Warriors
bowling_team                            Delhi Daredevils
batsman                                         AJ Finch
non_striker                                   RV Uthappa
bowler                                          M Morkel
runs_batsman                                           0
runs_extras                                            0
runs_total                                             0
dismissal_kind                                      None
Name: 10, dtype: object

In [173]:
initial_df.columns

Index(['match_id', 'inning', 'over', 'ball', 'batting_team', 'bowling_team',
       'batsman', 'non_striker', 'bowler', 'runs_batsman', 'runs_extras',
       'runs_total', 'dismissal_kind'],
      dtype='object')

In [149]:
# Checking the data 
first_innings = match['innings'][0]["1st innings"]
# print(first_innings)


In [150]:
# We will keep it simple and only look at Numerical columns
initial_df.describe()

,inning,over,ball,runs_batsman,runs_extras,runs_total
count,21522.000000,21522.000000,21522.000000,21522.000000,21522.000000,21522.000000
mean,1.479045,9.065050,9.426847,1.207183,0.070161,1.277344
std,0.499572,5.646762,5.646584,1.579061,0.355776,1.569651
min,1.000000,0.000000,0.100000,0.000000,0.000000,0.000000
25%,1.000000,4.000000,4.500000,0.000000,0.000000,0.000000
50%,1.000000,9.000000,9.300000,1.000000,0.000000,1.000000
75%,2.000000,14.000000,14.300000,1.000000,0.000000,1.000000
max,2.000000,19.000000,19.700000,6.000000,5.000000,7.000000


### To build a dream team (being biased), we need to analyse what was the highest/lowest score in an over, per batsman, highest run conceeded by a bowler, team average score etc. Once we figure out some simple stats we can move to complex data analysing by doing feature enginnering and thus add more columns.

Let's take a look at the winners how many times Delhi Daredevils won, and later find out win percentage.

In [151]:
initial_df.value_counts()


match_id                                  inning  over  ball  batting_team                 bowling_team      batsman         non_striker     bowler        runs_batsman  runs_extras  runs_total
2010-04-13, Brabourne Stadium             1       3     3.1   Mumbai Indians               Delhi Daredevils  SR Tendulkar    C Madan         PJ Sangwan    1             0            1             2
2008-04-19, Feroz Shah Kotla              1       0     0.1   Rajasthan Royals             Delhi Daredevils  T Kohli         YK Pathan       GD McGrath    0             0            0             1
2012-04-19, Feroz Shah Kotla              2       11    11.5  Delhi Daredevils             Deccan Chargers   KP Pietersen    IK Pathan       Ankit Sharma  0             0            0             1
                                                  12    12.6  Delhi Daredevils             Deccan Chargers   KP Pietersen    IK Pathan       JP Duminy     1             0            1             1
               

### Breaking the dataframe

Currently the dataframe contains everything from the $yaml$ files into single dataframe however as we observed above this is making hard to understand the data. Instead we will break the dataframes into two one will include info for matches and other ball by ball stat.


Keeping two clean, modular datasets gives you:

* Clarity during data exploration

* Flexibility for aggregation, modeling, and visualization

* Performance in terms of memory and speed

### *df_matches* 
One row per match. We will use this for match-level analysis, win percentage, win prediction, toss etc

### *df_deliveries* 
Ball by ball data. We will use this df for player performance and start feature engineering here first.

In [176]:
initial_df.columns

Index(['match_id', 'inning', 'over', 'ball', 'batting_team', 'bowling_team',
       'batsman', 'non_striker', 'bowler', 'runs_batsman', 'runs_extras',
       'runs_total', 'dismissal_kind'],
      dtype='object')

In [177]:
# Create match_id
# initial_df['match_id'] = (initial_df["match_date"].astype(str) + ", " + initial_df["venue"].astype(str))


# Create df_matches
# df_matches = initial_df[["match_id", 'match_date', 'venue', 'winner', 'player_of_match', 'toss_winner']].drop_duplicates()
# Create df_deliveries
# We will add match_id column to our new df so we could use it later if we need to merge it
df_deliveries_clean = initial_df[['match_id', 'inning', 'batting_team','bowling_team', 'over', 'ball',
                                  'batsman', 'non_striker', 'bowler',
                                  'runs_batsman', 'runs_extras', 
                                  'runs_total', 'dismissal_kind']].copy()


# Since we will be working with new dataframe a lot, it may cause error when we slice current df again and again

After creating the two new dataframes we will work on basic feature engineering first and create new columns.

In [178]:
df_deliveries_clean.tail()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,runs_batsman,runs_extras,runs_total,dismissal_kind
21517,"2008-05-11, Sawai Mansingh Stadium",2,Rajasthan Royals,Delhi Daredevils,18,18.3,M Rawat,Niraj Patel,MF Maharoof,0,0,0,None
21518,"2008-05-11, Sawai Mansingh Stadium",2,Rajasthan Royals,Delhi Daredevils,18,18.4,M Rawat,Niraj Patel,MF Maharoof,0,0,0,None
21519,"2008-05-11, Sawai Mansingh Stadium",2,Rajasthan Royals,Delhi Daredevils,18,18.5,M Rawat,Niraj Patel,MF Maharoof,1,0,1,None
21520,"2008-05-11, Sawai Mansingh Stadium",2,Rajasthan Royals,Delhi Daredevils,18,18.6,Niraj Patel,M Rawat,MF Maharoof,4,0,4,None
21521,"2008-05-11, Sawai Mansingh Stadium",2,Rajasthan Royals,Delhi Daredevils,19,19.1,M Rawat,Niraj Patel,VY Mahesh,4,0,4,None


### Feature Engineering

In [154]:
# Checking data types
print(df_deliveries_clean.dtypes)

match_id         object
inning            int64
batting_team     object
over              int64
ball            float64
batsman          object
non_striker      object
bowler           object
runs_batsman      int64
runs_extras       int64
runs_total        int64
dtype: object


Create column for 4's and 6's scored

In [159]:
# Column for 4's scored
df_deliveries_clean['is_four'] = (df_deliveries_clean['runs_batsman']==4).astype(int)
# Column for 6's scored
df_deliveries_clean['is_six'] = (df_deliveries_clean['runs_batsman']==6).astype(int)

# check the new cols
df_deliveries_clean.tail()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,runs_batsman,runs_extras,runs_total,is_four,is_six
21517,"2008-05-11, Sawai Mansingh Stadium",2,Rajasthan Royals,Delhi Daredevils,18,18.3,M Rawat,Niraj Patel,MF Maharoof,0,0,0,0,0
21518,"2008-05-11, Sawai Mansingh Stadium",2,Rajasthan Royals,Delhi Daredevils,18,18.4,M Rawat,Niraj Patel,MF Maharoof,0,0,0,0,0
21519,"2008-05-11, Sawai Mansingh Stadium",2,Rajasthan Royals,Delhi Daredevils,18,18.5,M Rawat,Niraj Patel,MF Maharoof,1,0,1,0,0
21520,"2008-05-11, Sawai Mansingh Stadium",2,Rajasthan Royals,Delhi Daredevils,18,18.6,Niraj Patel,M Rawat,MF Maharoof,4,0,4,1,0
21521,"2008-05-11, Sawai Mansingh Stadium",2,Rajasthan Royals,Delhi Daredevils,19,19.1,M Rawat,Niraj Patel,VY Mahesh,4,0,4,1,0


### We will now try to make the data lighter. Instead of working with 120* balls we will do an over by over summary. This will again help us in our further analysis

In [160]:
over_summary = (
    df_deliveries_clean.groupby(['match_id', 'inning', 'batting_team', 'over'])
    .agg(runs_in_over = ('runs_total','sum'),
        balls_bowled = ('ball', 'count'),
        fours = ('is_four', 'sum'),
        six = ('is_six', 'sum'),
        ).reset_index()
)
# Run rate
over_summary['run_rate'] = over_summary['runs_in_over']/over_summary['balls_bowled']*6
over_summary.head()

,match_id,inning,batting_team,over,runs_in_over,balls_bowled,fours,six,run_rate
0,"2008-04-19, Feroz Shah Kotla",1,Rajasthan Royals,0,7,7,0,1,6.0
1,"2008-04-19, Feroz Shah Kotla",1,Rajasthan Royals,1,10,6,2,0,10.0
2,"2008-04-19, Feroz Shah Kotla",1,Rajasthan Royals,2,0,6,0,0,0.0
3,"2008-04-19, Feroz Shah Kotla",1,Rajasthan Royals,3,14,6,3,0,14.0
4,"2008-04-19, Feroz Shah Kotla",1,Rajasthan Royals,4,6,6,1,0,6.0


### We hve created **$Over$ $Summary$** this helped deriving over by over run rate, to focus on team-innings. Powerplay overs, death overs(last overs) and more.
### We will now drill down further and focus on our dream team for Delhi Daredevils

In [161]:
team_innings_features = (
    over_summary
    .groupby(['match_id', 'inning', 'batting_team'])
    .agg(
        avg_run_rate=('run_rate', 'mean'),
        total_runs=('runs_in_over', 'sum'),
        total_fours=('fours', 'sum'),
        total_sixes=('six', 'sum'),
        high_scoring_overs=('runs_in_over', lambda x: (x >= 10).sum())
    )
    .reset_index()
)

### We will focus on individual player performance in two groups, $batsmen$ and $bowler$.
### *Batting stats*

In [162]:
df_deliveries_clean.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,runs_batsman,runs_extras,runs_total,is_four,is_six
0,"2013-05-19, Subrata Roy Sahara Stadium",1,Pune Warriors,Delhi Daredevils,0,0.1,RV Uthappa,AJ Finch,IK Pathan,0,0,0,0,0
1,"2013-05-19, Subrata Roy Sahara Stadium",1,Pune Warriors,Delhi Daredevils,0,0.2,RV Uthappa,AJ Finch,IK Pathan,2,0,2,0,0
2,"2013-05-19, Subrata Roy Sahara Stadium",1,Pune Warriors,Delhi Daredevils,0,0.3,RV Uthappa,AJ Finch,IK Pathan,4,0,4,1,0
3,"2013-05-19, Subrata Roy Sahara Stadium",1,Pune Warriors,Delhi Daredevils,0,0.4,RV Uthappa,AJ Finch,IK Pathan,0,0,0,0,0
4,"2013-05-19, Subrata Roy Sahara Stadium",1,Pune Warriors,Delhi Daredevils,0,0.5,RV Uthappa,AJ Finch,IK Pathan,1,0,1,0,0


In [163]:
# Filtering batting stats for Delhi Daredevils
dd_batting_stats = df_deliveries_clean[df_deliveries_clean['batting_team']=='Delhi Daredevils']
dd_batting_stats = (dd_batting_stats.groupby('batsman')
                 .agg(
                     matches_played = ('match_id', 'nunique'),
                     total_runs = ('runs_batsman', 'sum'),
                     balls_faced = ('ball', 'count'),
                     fours = ('is_four', 'sum'),
                     six = ('is_six', 'sum'),
                 )
                .reset_index()
)
## Deriving additional columns
## Average
dd_batting_stats['average'] = (dd_batting_stats['total_runs']/dd_batting_stats['balls_faced'])*10
## Strike rate
dd_batting_stats['strike_rate'] = ((dd_batting_stats['total_runs']/dd_batting_stats['balls_faced'])*100).round(decimals=1)
## Boundary percentage
dd_batting_stats['boundary_percentage'] = (((dd_batting_stats['fours'] + dd_batting_stats['six'])/dd_batting_stats['balls_faced']) *100).round(decimals=1)
# Checking the new df
dd_batting_stats

,batsman,matches_played,total_runs,balls_faced,fours,six,average,strike_rate,boundary_percentage
0,A Mishra,15,102,116,6,2,8.793103,87.9,6.9
1,A Nehra,7,25,29,2,1,8.620690,86.2,10.3
2,AB Agarkar,3,19,23,1,0,8.260870,82.6,4.3
3,AB Dinda,1,0,5,0,0,0.000000,0.0,0.0
4,AB McDonald,5,76,65,3,2,11.692308,116.9,7.7
...,...,...,...,...,...,...,...,...,...
56,V Sehwag,79,2174,1410,266,85,15.418440,154.2,24.9
57,VR Aaron,3,1,8,0,0,1.250000,12.5,0.0
58,VY Mahesh,5,15,28,0,0,5.357143,53.6,0.0
59,Y Nagar,20,285,271,20,9,10.516605,105.2,10.7


In [172]:
# Filtering bowling stats for Delhi Daredevils
dd_bowling_stats = df_deliveries_clean[df_deliveries_clean['bowling_team'] == 'Delhi Daredevils']
dd_bowling_stats = (dd_bowling_stats.groupby('bowler')
                   .agg(
                       matches_played = ('match_id', 'nunique'),
                       balls_bowled = ('ball', 'count'),
                       runs_conceeded = ('runs_total', 'sum'),
                       wickets=('dismissal_kind', lambda x: x.notna().sum())
                   )).reset_index()



KeyError: "Column(s) ['dismissal_kind'] do not exist"

In [137]:
# Total batsman used by Delhi daredevils in 92 matches
dd_batting_stats['batsman'].nunique()

61

A total of 61 batsman or players were used by Delhi Daredevils

### Let's create the bowler stats now


### We still lack one more column called **$Average$** for complete analysis. Since we do not have explicit column in the resource when the batsman got out, we will go ahead and try to make one.
Batting average =Total Runs Scored / Number of Times Dismissed

In [138]:
dd_batting_stats['average'].max()
dd_batting_stats['strike_rate'].max()
dd_batting_stats['boundary_percentage'].max()

40.0